# Import and run model

In [ ]:
import numpy as np
import tensorflow as tf
import keras
from sklearn.metrics import jaccard_score
from sklearn.model_selection import train_test_split


C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Import CIFAR10 dataset

In [ ]:

batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Convert images to greyscale
x_train = np.dot(x_train[..., :3], [0.2989, 0.5870, 0.1140])
x_test = np.dot(x_test[..., :3], [0.2989, 0.5870, 0.1140])

# Normalize data
x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 32, 32, 1))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 32, 32, 1))

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Convert labels to one-hot encoded format
num_classes = 10  # Replace with the number of classes in your problem
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

## Import model

In [ ]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="CNN_no_dense128_grey.tflite")
interpreter.allocate_tensors()

In [ ]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

## Model summary

In [ ]:
#print model architecture
tf.lite.experimental.Analyzer.analyze(model_path="CNN_no_dense128_grey.tflite",
                                      model_content=None,
                                      gpu_compatibility=False)

=== CNN_no_dense128_grey.tflite ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the QUANTIZE op takes
tensor #0 as input and produces tensor #7 as output.

Subgraph#0 main(T#0) -> [T#19]
  Op#0 QUANTIZE(T#0) -> [T#7]
  Op#1 CONV_2D(T#7, T#8[], T#2[1120, 476, -2147483648, 2194, 128, ...]) -> [T#9]
  Op#2 CONV_2D(T#9, T#10[], T#3[-1352, -14269, -13152, -12349, -12221, ...]) -> [T#11]
  Op#3 CONV_2D(T#11, T#12[], T#4[-4796, 117, -1993, -2843, -1339, ...]) -> [T#13]
  Op#4 MEAN(T#13, T#1[1, 2]) -> [T#14]
  Op#5 FULLY_CONNECTED(T#14, T#15[], T#6[-2055, -732, -1253, 122, 161, ...]) -> [T#16]
  Op#6 FULLY_CONNECTED(T#16, T#17[], T#5[-896, -194, 2165, 5900, -1256, ...]) -> [T#18]
  Op#7 DEQUANTIZE(T#18) -> [T#19]

Tensors of Subgraph#0
  T#0(serving_default_quantize_layer_input:0) shape_signature:[-1, 32, 32, 1], type:FLOAT32
  T#1(sequential_2/quant_global_average_pooling2d/Mean/reduction_indices) sha

## Test model with individual data sample

In [ ]:
test_sample = 99

input_shape = input_details[0]['shape']
interpreter.set_tensor(input_details[0]['index'], x_test[test_sample][np.newaxis, ...])
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[0.         0.         0.3737558  0.39953208 0.         0.
  0.         0.03866439 0.05155253 0.         0.64440656 0.5155253
  0.         0.         0.489749   0.         0.         0.
  0.         0.76039976 0.         0.07732879 0.         0.
  0.         0.         0.         0.14176945 0.         0.
  0.29642704 0.         0.5284134  0.489749   0.         0.
  0.         0.32220328 0.7732879  0.18043384 0.         0.5155253
  0.         0.         0.2835389  0.01288813 0.         0.
  0.         0.         0.         0.         0.15465757 0.57996595
  0.         0.         0.         0.         0.23198637 0.25776264
  0.         0.21909824 0.21909824 0.76039976 0.05155253 0.
  0.         0.         0.6057422  0.2448745  0.6186303  0.
  0.4510846  0.         0.         0.46397275 0.36086768 0.
  0.2062101  0.         0.36086768 0.8119523  0.         0.
  0.         0.         0.50263715 0.46397275 0.06444066 0.
  0.47686088 0.09021692 0.         0.         0.5670778  0.6186303
  0

In [ ]:
y_test[test_sample]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32)

# Extract Feature Embeddings

In [ ]:
x_train1000 = x_train[:1000]

In [ ]:
y_train1000 = y_train[:1000]

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import jaccard_score
from joblib import Parallel, delayed

def extract_feature_maps(interpreter, input_data):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    feature_maps = interpreter.get_tensor(output_details[0]['index'])
    return feature_maps

def extract_feature_maps_parallel(test_sample, model_path, index):
    test_sample = np.expand_dims(test_sample, axis=0)

    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    feature_maps = extract_feature_maps(interpreter, test_sample)
    print(f"Processed sample {index}")
    return feature_maps

def binarize_feature_maps(feature_maps, threshold=0.5):
    binary_maps = np.where(feature_maps > threshold, 1.0, 0.0)
    return binary_maps

def create_aggregate_embeddings(binary_maps, labels):
    num_classes = 10
    aggregate_embeddings = []

    for class_label in range(num_classes):
        class_indices = np.where(labels == class_label)[0]
        class_binary_maps = binary_maps[class_indices]
        class_embedding = np.mean(class_binary_maps, axis=0)
        aggregate_embeddings.append(class_embedding)

    aggregate_embeddings = np.stack(aggregate_embeddings)
    return aggregate_embeddings

test_feature_maps = []

model_path = "CNN_512_grey_depth.tflite"

# Extract feature maps for each test sample in parallel
print("Extracting feature maps for training samples...")
test_feature_maps = Parallel(n_jobs=-1)(delayed(extract_feature_maps_parallel)(test_sample, model_path, index) for index, test_sample in enumerate(x_train1000))
test_feature_maps = np.concatenate(test_feature_maps, axis=0)
print("Feature map extraction for training samples completed.")

# Binarize the test feature maps
test_binary_maps = binarize_feature_maps(test_feature_maps)

# Create aggregate embeddings for each class
test_labels = np.argmax(y_train1000, axis=1)
aggregate_embeddings = create_aggregate_embeddings(test_binary_maps, test_labels)

print("Aggregate Embeddings Shape:", aggregate_embeddings.shape)



Extracting feature maps for training samples...
Feature map extraction for training samples completed.
Aggregate Embeddings Shape: (10, 512)


In [ ]:
test_feature_maps2 = []
print("Extracting feature maps for test samples...")
test_feature_maps2 = Parallel(n_jobs=-1)(delayed(extract_feature_maps_parallel)(test_sample, model_path, index) for index, test_sample in enumerate(x_test1000))
test_feature_maps2 = np.concatenate(test_feature_maps2, axis=0)
print("Feature map extraction for test samples completed.")

# Binarize the test feature maps
test_binary_maps2 = binarize_feature_maps(test_feature_maps2)


Extracting feature maps for test samples...
Feature map extraction for test samples completed.


# Pattern Matching

In [ ]:
y_test1000 = y_test[:1000]

In [ ]:
def jaccard_similarity(reference_embedding, query_embedding):
    intersection = tf.reduce_sum(tf.minimum(reference_embedding, query_embedding))
    union = tf.reduce_sum(tf.maximum(reference_embedding, query_embedding))
    jaccard_score = intersection / union
    return jaccard_score

def predict_class(reference_embeddings, query_embedding):
    jaccard_scores = []
    for reference_embedding in reference_embeddings:
        score = jaccard_similarity(reference_embedding, query_embedding)
        jaccard_scores.append(score)
    predicted_class = tf.argmax(jaccard_scores).numpy()
    return predicted_class

# Binarize the aggregate embeddings
binary_aggregate_embeddings = binarize_feature_maps(aggregate_embeddings)

# Predict the class for each test sample
predicted_labels = []
for query_embedding in test_binary_maps2:
    predicted_class = predict_class(aggregate_embeddings, query_embedding)
    predicted_labels.append(predicted_class)

# Convert test labels to 1D array
test_labels = np.argmax(y_test1000, axis=1)

# Convert predicted labels to numpy array
predicted_labels = np.array(predicted_labels)

# Calculate accuracy
correct_predictions = (predicted_labels == test_labels).sum()
accuracy = correct_predictions / len(test_labels)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.1030


In [ ]:
def binarize_embeddings(embeddings, threshold):
    binary_embeddings = np.where(embeddings > threshold, 1, 0)
    return binary_embeddings

# Binarize the aggregate embeddings
binary_aggregate_embeddings = binarize_embeddings(aggregate_embeddings, threshold=0.5)

# Predict the class for each test sample using Jaccard similarity
predicted_labels_jaccard = []
for query_embedding in test_binary_maps2:
    jaccard_scores = []
    for reference_embedding in binary_aggregate_embeddings:
        score = jaccard_score(reference_embedding, query_embedding)
        jaccard_scores.append(score)
    predicted_class = np.argmax(jaccard_scores)
    predicted_labels_jaccard.append(predicted_class)

# Convert predicted labels to numpy array
predicted_labels_jaccard = np.array(predicted_labels_jaccard)

# Calculate accuracy using Jaccard similarity
correct_predictions_jaccard = (predicted_labels_jaccard == test_labels).sum()
accuracy_jaccard = correct_predictions_jaccard / len(test_labels)
print(f"Accuracy (Jaccard Similarity): {accuracy_jaccard:.4f}")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\skl

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\skl

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\skl

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\skl

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\skl

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\skl

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\skl

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\skl

Accuracy (Jaccard Similarity): 0.1030


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\Anaconda3\lib\site-packages\skl

In [ ]:
import numpy as np

def quantize_embeddings(embeddings, num_bits=2):
    num_levels = 2 ** num_bits
    min_value = np.min(embeddings)
    max_value = np.max(embeddings)

    # Normalize the embeddings to the range [0, 1]
    normalized_embeddings = (embeddings - min_value) / (max_value - min_value)

    # Quantize the embeddings to num_levels
    quantized_embeddings = np.round(normalized_embeddings * (num_levels - 1))

    return quantized_embeddings

# Quantize the aggregate embeddings to 2-bit representations
quantized_aggregate_embeddings = quantize_embeddings(aggregate_embeddings, num_bits=2)

# Quantize the test embeddings to 2-bit representations
quantized_test_embeddings = quantize_embeddings(test_binary_maps2, num_bits=2)

# Predict the class for each test sample using Jaccard similarity
predicted_labels_jaccard = []
for query_embedding in quantized_test_embeddings:
    jaccard_scores = []
    for reference_embedding in quantized_aggregate_embeddings:
        intersection = np.sum(np.minimum(reference_embedding, query_embedding))
        union = np.sum(np.maximum(reference_embedding, query_embedding))
        jaccard_score = intersection / union
        jaccard_scores.append(jaccard_score)
    predicted_class = np.argmax(jaccard_scores)
    predicted_labels_jaccard.append(predicted_class)

# Convert predicted labels to numpy array
predicted_labels_jaccard = np.array(predicted_labels_jaccard)

# Calculate accuracy using Jaccard similarity
correct_predictions_jaccard = (predicted_labels_jaccard == test_labels).sum()
accuracy_jaccard = correct_predictions_jaccard / len(test_labels)
print(f"Accuracy (Jaccard Similarity with 2-bit embeddings): {accuracy_jaccard:.4f}")

Accuracy (Jaccard Similarity with 2-bit embeddings): 0.1030


C:\Users\CMP3WO~1\AppData\Local\Temp/ipykernel_31652/453081587.py:9: RuntimeWarning: invalid value encountered in divide
  normalized_embeddings = (embeddings - min_value) / (max_value - min_value)


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

def cosine_similarity_score(reference_embedding, query_embedding):
    similarity = cosine_similarity(reference_embedding.reshape(1, -1), query_embedding.reshape(1, -1))
    return similarity[0][0]

def euclidean_distance_score(reference_embedding, query_embedding):
    distance = euclidean_distances(reference_embedding.reshape(1, -1), query_embedding.reshape(1, -1))
    return 1 / (1 + distance[0][0])  # Convert distance to similarity score

def predict_class_cosine(reference_embeddings, query_embedding):
    cosine_scores = []
    for reference_embedding in reference_embeddings:
        score = cosine_similarity_score(reference_embedding, query_embedding)
        cosine_scores.append(score)
    predicted_class = np.argmax(cosine_scores)
    return predicted_class

def predict_class_euclidean(reference_embeddings, query_embedding):
    euclidean_scores = []
    for reference_embedding in reference_embeddings:
        score = euclidean_distance_score(reference_embedding, query_embedding)
        euclidean_scores.append(score)
    predicted_class = np.argmax(euclidean_scores)
    return predicted_class

# Predict the class for each test sample using cosine similarity
predicted_labels_cosine = []
for query_embedding in test_binary_maps2:
    predicted_class = predict_class_cosine(aggregate_embeddings, query_embedding)
    predicted_labels_cosine.append(predicted_class)

# Predict the class for each test sample using Euclidean distance
predicted_labels_euclidean = []
for query_embedding in test_binary_maps2:
    predicted_class = predict_class_euclidean(aggregate_embeddings, query_embedding)
    predicted_labels_euclidean.append(predicted_class)

# Convert test labels to 1D array
test_labels = np.argmax(y_test1000, axis=1)

# Convert predicted labels to numpy arrays
predicted_labels_cosine = np.array(predicted_labels_cosine)
predicted_labels_euclidean = np.array(predicted_labels_euclidean)

# Calculate accuracy using cosine similarity
correct_predictions_cosine = (predicted_labels_cosine == test_labels).sum()
accuracy_cosine = correct_predictions_cosine / len(test_labels)
print(f"Accuracy (Cosine Similarity): {accuracy_cosine:.4f}")

# Calculate accuracy using Euclidean distance
correct_predictions_euclidean = (predicted_labels_euclidean == test_labels).sum()
accuracy_euclidean = correct_predictions_euclidean / len(test_labels)
print(f"Accuracy (Euclidean Distance): {accuracy_euclidean:.4f}")

Accuracy (Cosine Similarity): 0.1030
Accuracy (Euclidean Distance): 0.1030


## Test with individual samples

In [ ]:
binary_aggregate_embeddings = binarize_feature_maps(aggregate_embeddings)

# Predict the class of a single test sample using Jaccard score
test_sample_index = 35
test_sample_embedding = test_binary_maps[test_sample_index]
test_sample_label = test_labels[test_sample_index]

jaccard_scores = []
for class_embedding in binary_aggregate_embeddings:
    jaccard_scores.append(jaccard_score(test_sample_embedding.flatten(), class_embedding.flatten(), average='micro'))

predicted_class = np.argmax(jaccard_scores)

print("Test Sample Index:", test_sample_index)
print("True Label:", test_sample_label)
print("Predicted Class:", predicted_class)
print("Jaccard Scores:", jaccard_scores)

Test Sample Index: 35
True Label: 2
Predicted Class: 0
Jaccard Scores: [0.9393939393939394, 0.8823529411764706, 0.9393939393939394, 0.9393939393939394, 0.8823529411764706, 0.9393939393939394, 0.8823529411764706, 0.8823529411764706, 0.9393939393939394, 0.8285714285714286]


In [ ]:
 test_binary_maps[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Predict the class of a single test sample using cosine similarity
test_sample_index = 35
test_sample_embedding = test_binary_maps[test_sample_index]
test_sample_label = test_labels[test_sample_index]

cosine_similarities = []
for class_embedding in aggregate_embeddings:
    similarity = cosine_similarity(test_sample_embedding.reshape(1, -1), class_embedding.reshape(1, -1))
    cosine_similarities.append(similarity[0][0])

predicted_class = np.argmax(cosine_similarities)

print("Test Sample Index:", test_sample_index)
print("True Label:", test_sample_label)
print("Predicted Class:", predicted_class)
print("Cosine Similarities:", cosine_similarities)

Test Sample Index: 35
True Label: 2
Predicted Class: 8
Cosine Similarities: [0.9605019364570001, 0.951316144768461, 0.9495775577040491, 0.9522707843312521, 0.9314171138397463, 0.9452544821727449, 0.9263015447839666, 0.934634210247262, 0.9812515338297501, 0.9511295480996489]
